In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from skimage import io, color, img_as_bool, exposure, transform, feature
%matplotlib inline

In [52]:
# file paths
ci_path = '/home/eydu/plane_data/cropped_images_18/'
fi_path = '/home/eydu/plane_data/full_images_18/'
l_file = '/home/eydu/plane_data/plane_labels_2018.csv'
sandbox = '/home/eydu/plane_data/sandbox/'

In [5]:
# all labels
plane_data = pd.read_csv(l_file)
print(plane_data.head())
print(plane_data.shape)

                   img_name  plane
0  2016-08-02T13+50+24_430Z  False
1  2016-08-02T14+12+37_390Z  False
2  2016-08-02T22+20+26_600Z  False
3  2016-08-03T12+04+30_670Z  False
4  2016-08-03T12+32+21_790Z  False
(6758, 2)


In [6]:
# filter to include only the airplane images
print(plane_data['plane'].sum())
aircraft = plane_data[plane_data['plane']==True]
print(aircraft.shape)

101
(101, 2)


In [47]:
# Run once to prep data
# NOTE: SPLIT THESE INTO TRAIN TEST BATCHES!
# mlp: hidden neurons, lazers, iters

posImg = plane_data[plane_data['plane']==True]
negImg = plane_data[plane_data['plane']==False]

fp = open(sandbox + "info.txt","w") 
fn = open(sandbox + "bg.txt","w") 

# NOTE MIGHT NEED TO GEN MORE FILES!  Typ: 1000.  HAVE: 101
for index, row in posImg.iterrows():
    imname = ci_path + row['img_name'] + '.png'
    img_raw = cv2.imread(imname)
    # final_image = color.rgb2gray(img_raw)
    final_image = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
    im = sandbox + 'pos/' + row['img_name'] + '.png'
    plt.imsave(im, final_image)
    fp.write('pos/' + row['img_name'] + '.png 1 0 0 140 60\n')
    
for index, row in negImg.iterrows():
    imname = ci_path + row['img_name'] + '.png'
    img_raw = cv2.imread(imname)
    # final_image = color.rgb2gray(img_raw)
    final_image = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
    im = sandbox + 'neg/' + row['img_name'] + '.png'
    plt.imsave(im, final_image)
    fn.write('neg/' + row['img_name'] + '.png\n')
    
fp.close()
fn.close()

In [ ]:
# Generate needed files

# in sandbox
# Vector File (stitching all positives together)
# opencv_createsamples-info info.txt -w 20 -h 20 -vec pos.vec -num 101 # might up this to 25x25 but increases work exponentially
# opencv_traincascade -data haarData -vec pos.vec -bg bg.txt -numPos 101, -numNeg 6657 -numStages 10 -w 20 -h 20

# makePos.sh
# find info/* -name *.jpg | wc -l # to verify the number

# ATTIC
# opencv_createsamples -img ourimg -bg bg.txt -info info/info.lst -pngoutput info -maxxangle 0.5 -maxyangle 0.5 -maxzangle 0.5 -num 30
# find pos/ -exec opencv_createsamples -img {} -bg bg.txt -info info/info.lst -pngoutput info -maxxangle 0.5 -maxyangle 0.5 -maxzangle 0.5 -num 3 \; -exec bash -c "cat info/info.lst >> info/info.txt" \;

In [50]:
import cv2

#LBP CASCADE WORKS GREAT

X = plane_data

plane_cascade = cv2.CascadeClassifier('/home/eydu/plane_data/sandbox/haarData/cascade.xml')

def haarPredict(X):
    rv = []
    
    for index, row in X.iterrows():
        imname = fi_path + row['img_name'] + '.png'
        img_raw = cv2.imread(imname)
        #final_image = color.rgb2gray(img_raw)
        final_image = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY) # instead of converting io.imread conversion
        # final_image0 = np.array(final_image, dtype='uint8')
        rv.append(len(plane_cascade.detectMultiScale(final_image, 50, 50)) > 0)
        #rv.append(len(plane_cascade.detectMultiScale(final_image0)) > 0)
        
    return(np.array(rv))


In [53]:
from my_measures import BinaryClassificationPerformance

y = plane_data['plane'] # get vector of labels 


performance = BinaryClassificationPerformance(haarPredict(X), y, 'haar wavelet cascade')
performance.compute_measures()
print(performance.performance_measures)

{'Pos': 101, 'Neg': 6657, 'TP': 0, 'TN': 6656, 'FP': 1, 'FN': 101, 'Accuracy': 0.9849067771530039, 'Precision': 0.0, 'Recall': 0.0, 'desc': 'haar wavelet cascade'}


In [5]:
# take a random subset of 20 of the airplane images
aircraft_subset = aircraft.sample(20)
print(aircraft_subset.shape)

(20, 2)


In [8]:
# for each of the subset images, alter them and view the alterations
f = open(sandbox + "view_images.html","w") 
first_image = False

for index, row in aircraft_subset.iterrows():
    imname = ci_path + row['img_name'] + '.png'
    img_raw = io.imread(imname)
#     final_image = img_as_bool(img_raw) 
    final_image = color.rgb2gray(img_raw)
    im = sandbox + row['img_name'] + '.png'
    plt.imsave(im, final_image)
    if first_image==False:
        print(final_image.shape)
        print(final_image)
        first_image = True
    f.write("<img src='" + row['img_name'] + ".png' width='320'>\n")

f.close() 


(60, 140)
[[ 0.41239137  0.37317569  0.20454824 ...,  0.3607902   0.35294706
   0.31178196]
 [ 0.39668235  0.37904667  0.23394863 ...,  0.56724941  0.58293569
   0.56641608]
 [ 0.40452549  0.37904667  0.24179176 ...,  0.44175922  0.41038667
   0.40955333]
 ..., 
 [ 0.52405373  0.52405373  0.52013216 ...,  0.47503059  0.47503059
   0.47503059]
 [ 0.52013216  0.52013216  0.51621059 ...,  0.47895216  0.47895216
   0.47895216]
 [ 0.52013216  0.52013216  0.51621059 ...,  0.49071686  0.49071686
   0.49071686]]
